In [1]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import warnings
import requests
import pandas as pd
import json
from pandas import json_normalize
from psycopg2.extras import RealDictCursor
import psycopg2 as psycopg2
warnings.filterwarnings('ignore')



%matplotlib inline


In [2]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv('sample.env')

True

In [34]:
def connect_dB():
    #connect to cloudSQL 
    uri = "postgres://avnadmin:aivencloud.com:24670/defaultdb?sslmode=require"
    db_conn = psycopg2.connect(uri)
    return db_conn
    
def createTable(db_conn):
     c = db_conn.cursor(cursor_factory=RealDictCursor)
     c.execute("CREATE TABLE polygon_market_data(symbol VARCHAR(5),time_stamp bigint,high FLOAT,low FLOAT,open FLOAT, close FLOAT)")
     c.close()
    
def insertMarketValue(db_conn, ticker, json_data):
    c = db_conn.cursor(cursor_factory=RealDictCursor)
    for results in json_data['results']:
        c.execute ('INSERT INTO polygon_market_data(symbol, time_stamp, high, low, open, close) values (%s,%s,%s,%s,%s,%s)',(ticker,results['t'], results['h'],results['l'],results['o'],results['c']))
        db_conn.commit()
    c.execute("SELECT count(*) from polygon_market_data;");
    rows = c.fetchall()
    print(rows)
      
            
def tearDown(db_conn):
    c = db_conn.cursor(cursor_factory=RealDictCursor)
    c.execute("DROP TABLE polygon_market_data")
    
    c.close()
    db_conn.close()
  

In [35]:
# Set the variables for the Polygon API and secret keys
polygon_api_key = os.getenv("POLYGON_API_SECRET_KEY")
polygon_name= os.getenv("NAME")

display(polygon_api_key)
display(polygon_name)


'yb2NKhw_VkcEhSR7lj_49mvoQXbO0rPH'

'Default'

In [36]:
# Call Polygon Get API for the Ticker Requested
def retrieve_Market_data(symbol, range, start_date, end_date): 
    
    #connect to dB
    conn = connect_dB()
    ##createTable(conn[0], conn[1])
    
    #connect to polygon API
    URL = 'https://api.polygon.io/v2/aggs/ticker/'+ symbol +'/range/'+range+'/day/'+start_date+'/'+end_date+'?adjusted=true&sort=asc&limit=120&apiKey='+polygon_api_key
    session = requests.Session()
    response = session.get(URL)
    
    #write to db
    if (response.status_code == 200):
        market_data = response.json()
        json_data = json.loads(response.text)
        insertMarketValue(conn,  symbol, json_data)                  
     #   display(market_data)
        display(json_data['results'])
        tearDown(conn)
    
    else: 
        print("Polygon API Errored")
 


In [38]:
start_date = "2022-01-01"
end_date = "2022-02-01"
response_data = retrieve_Market_data("AAPL", "1",start_date, end_date )



[RealDictRow([('count', 191)])]


[{'v': 104677470.0,
  'vw': 181.4156,
  'o': 177.83,
  'c': 182.01,
  'h': 182.88,
  'l': 177.71,
  't': 1641186000000,
  'n': 772691},
 {'v': 99110438.0,
  'vw': 180.5574,
  'o': 182.63,
  'c': 179.7,
  'h': 182.94,
  'l': 179.12,
  't': 1641272400000,
  'n': 831890},
 {'v': 94535602.0,
  'vw': 177.2884,
  'o': 179.61,
  'c': 174.92,
  'h': 180.17,
  'l': 174.64,
  't': 1641358800000,
  'n': 848513},
 {'v': 96882954.0,
  'vw': 173.08,
  'o': 172.7,
  'c': 172,
  'h': 175.3,
  'l': 171.64,
  't': 1641445200000,
  'n': 960340},
 {'v': 86709147.0,
  'vw': 172.4383,
  'o': 172.89,
  'c': 172.17,
  'h': 174.14,
  'l': 171.03,
  't': 1641531600000,
  'n': 716881},
 {'v': 106754551.0,
  'vw': 170.2886,
  'o': 169.08,
  'c': 172.19,
  'h': 172.5,
  'l': 168.17,
  't': 1641790800000,
  'n': 956337},
 {'v': 76138312.0,
  'vw': 173.6337,
  'o': 172.32,
  'c': 175.08,
  'h': 175.18,
  'l': 170.82,
  't': 1641877200000,
  'n': 649652},
 {'v': 74805173.0,
  'vw': 175.8343,
  'o': 176.12,
  'c': 175